In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import metpy.calc as mpcalc
import numpy as np
import xarray as xr
import glob
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pandas as pd
import cmocean.cm as cmo

In [2]:
# get dataset 
lat_min, lat_max = (20,90) #
lon_min, lon_max = (-80,70)


nao = np.load("/work/uo1075/u241321/data/eemd_t310_assi_Norwegian.npy")
NAO = nao[2,:] # remove non-linear trend and standardize 
y = NAO

data_x = xr.open_dataset('/work/uo1075/u241321/data/hfx_1970-2019_assi_dt.nc', decode_times=False)  # unit: w
data_y = xr.open_dataset('/work/uo1075/u241321/data/hfy_1970-2019_assi_dt.nc', decode_times=False)

var_x = np.mean(data_x['__xarray_dataarray_variable__'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max)), axis=1) 
var_y = np.mean(data_y['__xarray_dataarray_variable__'].sel(lat=slice(lat_min, lat_max), lon=slice(lon_min, lon_max)), axis=1) 




In [3]:
# regression onto x and y transport respectively, then calculate manitude with direction

field_x = var_x.stack(spatial=('lat','lon')).dropna(dim="spatial") #time,space

nn = 6 # number of regression year


from sklearn.linear_model import LinearRegression
def regression(x,y):

    coef = LinearRegression(fit_intercept=True).fit(x.reshape(-1, 1), y.values.reshape(-1, 1)).coef_
    

    return coef


# regression, center on 4-47, 44 year (start from 0)

coe_x = np.zeros((nn, field_x.shape[1]))

for m in range(0,field_x.shape[1],1):
        coe_x[0,m] = regression(y[5:49], field_x[0:44,m])
        coe_x[1,m] = regression(y[5:49], field_x[1:45,m])
        coe_x[2,m] = regression(y[5:49], field_x[2:46,m])
        coe_x[3,m] = regression(y[5:49], field_x[3:47,m])
        coe_x[4,m] = regression(y[5:49], field_x[4:48,m])
        coe_x[5,m] = regression(y[5:49], field_x[5:49,m])
       


In [4]:
coe_x = xr.DataArray(coe_x,  
                    dims=['mode','spatial'],
                    coords=dict(
                        spatial=field_x.spatial,
                         mode=np.arange(1,nn+1,1))
                    , )
# field = var.stack(spatial=('lat','lon')).dropna(dim="spatial") #time,space
spatial = field_x .coords["spatial"]
mode = coe_x .coords["mode"]
reg_x = xr.DataArray(coe_x, dims = ["mode","spatial"], coords = {"mode":mode,"spatial":spatial}).unstack()  


In [5]:
field_y = var_y.stack(spatial=('lat','lon')).dropna(dim="spatial") #time,space
coe_y = np.zeros((nn, field_y.shape[1]))

for m in range(0,field_y.shape[1],1):
        coe_y[0,m] = regression(y[5:49], field_y[0:44,m])
        coe_y[1,m] = regression(y[5:49], field_y[1:45,m])
        coe_y[2,m] = regression(y[5:49], field_y[2:46,m])
        coe_y[3,m] = regression(y[5:49], field_y[3:47,m])
        coe_y[4,m] = regression(y[5:49], field_y[4:48,m])
        coe_y[5,m] = regression(y[5:49], field_y[5:49,m])


In [6]:
coe_y = xr.DataArray(coe_y,  
                    dims=['mode','spatial'],
                    coords=dict(
                        spatial=field_y.spatial,
                         mode=np.arange(1,nn+1,1))
                    , )

spatial_y = field_y .coords["spatial"]
reg_y = xr.DataArray(coe_y, dims = ["mode","spatial"], coords = {"mode":mode,"spatial":spatial_y}).unstack()        
lon = reg_x.lon
lat = reg_x.lat

In [7]:
x = reg_x.stack(spatial=('mode','lat','lon'))
y = reg_y.stack(spatial=('mode','lat','lon'))
spatial = x.coords["spatial"]

In [8]:
from multiprocessing import Pool
from itertools import product
from itertools import starmap

def mask_insignificant(data1, data2):
    
    mode1 = np.where(np.sqrt(data1*data1+data2*data2) >= 1e+13 , data1, np.NaN )
    mode2 = np.where(np.sqrt(data1*data1+data2*data2) >= 1e+13, data2, np.NaN )
    
    return mode1, mode2


def grid_mask(m):
    
    mode = mask_insignificant(x[m], y[m])
    
    return mode


res = Pool().map(grid_mask,np.arange(0,x.shape[0],1))
re = np.array(res)
# output 
reg_transx = xr.DataArray(re[:,0], dims = ["spatial"], coords = {"spatial":spatial}).unstack()    
reg_transy = xr.DataArray(re[:,1], dims = ["spatial"], coords = {"spatial":spatial}).unstack()    

In [10]:
reg_transx.to_netcdf("/work/uo1075/u241321/data/reg_transx_T_c2_44_mask.nc")
reg_transy.to_netcdf("/work/uo1075/u241321/data/reg_transy_T_c2_44_mask.nc")